In [ ]:
import requests, pandas as pd
import os
from pathlib import Path
from google.cloud import storage
from google.oauth2 import service_account
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

ROOT = Path.cwd().parent

# Dossier de téléchargement des données
DATA_DIR = ROOT.joinpath("data")
DATA_DIR.mkdir(exist_ok=True)

# Configuration Google Cloud
PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = Path.cwd().parent.joinpath(Path(os.getenv("GOOGLE_APPLICATION_CREDENTIALS")))
BUCKET_NAME = os.getenv("BUCKET_NAME")


In [38]:
# Authentification Google Cloud
creds = service_account.Credentials.from_service_account_file(SA_PATH)
storage_client = storage.Client(project=PROJECT_ID, credentials=creds)

In [39]:
# Récupérer la liste des fichiers ZIP disponibles via l'API IDFM
URL = "https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/histo-validations-reseau-ferre/records"
print("Récupération de la liste des fichiers disponibles...")
r = requests.get(URL, timeout=60)
r.raise_for_status()

response_data = r.json()
records = response_data.get("results", [])
print(f"Nombre de fichiers trouvés: {len(records)}")

# Créer un DataFrame avec les métadonnées
data = []
for record in records:
    data.append({
        'annee': int(record.get('annee')),
        'filename': record.get('reseau_ferre', {}).get('filename'),
        'url': record.get('reseau_ferre', {}).get('url')
    })

df_metadata = pd.DataFrame(data)
print("\nFichiers disponibles:")
display(df_metadata)

Récupération de la liste des fichiers disponibles...
Nombre de fichiers trouvés: 10

Fichiers disponibles:


,annee,filename,url
0,2017,data-rf-2017.zip,https://data.iledefrance-mobilites.fr/api/expl...
1,2019,data-rf-2019.zip,https://data.iledefrance-mobilites.fr/api/expl...
2,2020,data-rf-2020.zip,https://data.iledefrance-mobilites.fr/api/expl...
3,2021,data-rf-2021.zip,https://data.iledefrance-mobilites.fr/api/expl...
4,2016,data-rf-2016.zip,https://data.iledefrance-mobilites.fr/api/expl...
5,2018,data-rf-2018.zip,https://data.iledefrance-mobilites.fr/api/expl...
6,2022,data-rf-2022.zip,https://data.iledefrance-mobilites.fr/api/expl...
7,2023,data-rf-2023.zip,https://data.iledefrance-mobilites.fr/api/expl...
8,2015,data-rf-2015.zip,https://data.iledefrance-mobilites.fr/api/expl...
9,2024,data-rf-2024.zip,https://data.iledefrance-mobilites.fr/api/expl...


In [40]:
import requests
from zipfile import ZipFile

# Base directory where zips are saved and extracted
BASE_DIR = DATA_DIR / "histo-validations-reseau-ferre"
BASE_DIR.mkdir(parents=True, exist_ok=True)

for _, row in df_metadata.iterrows():
    year = row["annee"]
    file_url = row["url"]
    file_name = row["filename"]

    # Créer un dossier pour l'année
    year_dir = BASE_DIR / str(year)
    year_dir.mkdir(exist_ok=True)
    
    # Chemin complet du fichier ZIP à télécharger
    zip_file_path = year_dir / file_name

    print(f"Téléchargement de {year} ({file_name})...")
    try:
        resp = requests.get(file_url, timeout=300)
        resp.raise_for_status()
        
        # Écrire le fichier ZIP
        with open(zip_file_path, "wb") as f:
            f.write(resp.content)
        
        print(f"Téléchargé: {zip_file_path}")
        print(f"Extraction de {file_name}...")
        
        # Extraire le ZIP
        with ZipFile(zip_file_path, "r") as zf:
            zf.extractall(year_dir)
        
        print(f"Extrait dans: {year_dir}")
        
        # Supprimer le fichier ZIP après extraction
        zip_file_path.unlink()
        print(f"Fichier ZIP supprimé")
        
    except Exception as e:
        print(f"Erreur pour {year}: {e}")
        continue

print(f"\nTéléchargement et extraction terminés dans: {BASE_DIR}")


Téléchargement de 2017 (data-rf-2017.zip)...
Téléchargé: c:\Users\oettouileb\core_reims\m2-univ-reims-sep-cs-etl-sncf-gcp\data\histo-validations-reseau-ferre\2017\data-rf-2017.zip
Extraction de data-rf-2017.zip...
Extrait dans: c:\Users\oettouileb\core_reims\m2-univ-reims-sep-cs-etl-sncf-gcp\data\histo-validations-reseau-ferre\2017
Fichier ZIP supprimé
Téléchargement de 2019 (data-rf-2019.zip)...
Téléchargé: c:\Users\oettouileb\core_reims\m2-univ-reims-sep-cs-etl-sncf-gcp\data\histo-validations-reseau-ferre\2019\data-rf-2019.zip
Extraction de data-rf-2019.zip...
Extrait dans: c:\Users\oettouileb\core_reims\m2-univ-reims-sep-cs-etl-sncf-gcp\data\histo-validations-reseau-ferre\2019
Fichier ZIP supprimé
Téléchargement de 2020 (data-rf-2020.zip)...
Téléchargé: c:\Users\oettouileb\core_reims\m2-univ-reims-sep-cs-etl-sncf-gcp\data\histo-validations-reseau-ferre\2020\data-rf-2020.zip
Extraction de data-rf-2020.zip...
Extrait dans: c:\Users\oettouileb\core_reims\m2-univ-reims-sep-cs-etl-sncf-g

In [42]:
def delete_zip_files(folder_path):
    """
    Fonction simple pour supprimer tous les fichiers ZIP dans un dossier et ses sous-dossiers
    
    Args:
        folder_path (str): Chemin du dossier à nettoyer
    """
    print(f"🗑️ Suppression des fichiers ZIP dans {folder_path}")
    
    folder = Path(folder_path)
    deleted_files = 0
    
    # Parcourir tous les fichiers ZIP dans le dossier et ses sous-dossiers
    for zip_file in folder.rglob("*.zip"):
        try:
            zip_file.unlink()  # Supprimer le fichier
            deleted_files += 1
            print(f"  ✅ Supprimé: {zip_file.relative_to(folder)}")
        except Exception as e:
            print(f"  ❌ Erreur lors de la suppression de {zip_file}: {e}")
    
    print(f"\n🎉 Nettoyage terminé! {deleted_files} fichiers ZIP supprimés")

# Utiliser la fonction pour nettoyer le dossier
data_folder = DATA_DIR / "histo-validations-reseau-ferre"
delete_zip_files(data_folder)


🗑️ Suppression des fichiers ZIP dans c:\Users\oettouileb\core_reims\m2-univ-reims-sep-cs-etl-sncf-gcp\data\histo-validations-reseau-ferre
  ✅ Supprimé: 2020\data-rf-2015.zip
  ✅ Supprimé: 2020\data-rf-2016.zip
  ✅ Supprimé: 2020\data-rf-2017.zip
  ✅ Supprimé: 2020\data-rf-2018.zip
  ✅ Supprimé: 2020\data-rf-2019.zip

🎉 Nettoyage terminé! 5 fichiers ZIP supprimés


In [43]:
# Fonction simple pour uploader le dossier vers GCS
def upload_folder_to_gcs(local_folder_path, bucket_name, gcs_folder_path):
    """
    Fonction simple pour uploader un dossier local vers Google Cloud Storage
    """
    print(f"Upload du dossier {local_folder_path} vers gs://{bucket_name}/{gcs_folder_path}")
    
    # Récupérer le bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Parcourir tous les fichiers dans le dossier local
    local_path = Path(local_folder_path)
    uploaded_files = 0
    
    for file_path in local_path.rglob("*"):  # rglob pour parcourir récursivement
        if file_path.is_file():
            # Calculer le chemin relatif
            relative_path = file_path.relative_to(local_path)
            
            # Créer le chemin de destination dans GCS
            gcs_file_path = f"{gcs_folder_path}/{relative_path}".replace("\\", "/")
            
            # Upload le fichier
            blob = bucket.blob(gcs_file_path)
            blob.upload_from_filename(file_path)
            
            uploaded_files += 1
            print(f"  ✅ {relative_path} → {gcs_file_path}")
    
    print(f"\n🎉 Upload terminé! {uploaded_files} fichiers uploadés vers gs://{bucket_name}/{gcs_folder_path}")

# Utiliser la fonction pour uploader le dossier
local_data_folder = DATA_DIR / "histo-validations-reseau-ferre"
gcs_destination = "bronze/histo-validations-reseau-ferre"

upload_folder_to_gcs(local_data_folder, BUCKET_NAME, gcs_destination)

Upload du dossier c:\Users\oettouileb\core_reims\m2-univ-reims-sep-cs-etl-sncf-gcp\data\histo-validations-reseau-ferre vers gs://bronze-sncf-etl/bronze/histo-validations-reseau-ferre
  ✅ 2015\data-rf-2015\2015S1_NB_FER.csv → bronze/histo-validations-reseau-ferre/2015/data-rf-2015/2015S1_NB_FER.csv
  ✅ 2015\data-rf-2015\2015S1_PROFIL_FER.csv → bronze/histo-validations-reseau-ferre/2015/data-rf-2015/2015S1_PROFIL_FER.csv
  ✅ 2015\data-rf-2015\2015S2_NB_FER.csv → bronze/histo-validations-reseau-ferre/2015/data-rf-2015/2015S2_NB_FER.csv
  ✅ 2015\data-rf-2015\2015S2_PROFIL_FER.csv → bronze/histo-validations-reseau-ferre/2015/data-rf-2015/2015S2_PROFIL_FER.csv
  ✅ 2016\data-rf-2016\2016S1_NB_FER.txt → bronze/histo-validations-reseau-ferre/2016/data-rf-2016/2016S1_NB_FER.txt
  ✅ 2016\data-rf-2016\2016S1_PROFIL_FER.txt → bronze/histo-validations-reseau-ferre/2016/data-rf-2016/2016S1_PROFIL_FER.txt
  ✅ 2016\data-rf-2016\2016S2_NB_FER.txt → bronze/histo-validations-reseau-ferre/2016/data-rf-2016